In this notebook, we will narrow down artists tour dates to those with enough information and make an analysis dataset.

In [1]:
import pandas as pd
import json
import os
import numpy as np

In [2]:
# get all files of interest
directory = "../data/source/salt_shed_artists/"
files = [file for file in os.listdir(directory) if "json" in file]

In [3]:
def clean_dates(json_data, folder="../data/processed/salt_shed_artists/", venue="The Salt Shed", days=45, outlier="std"):
    # first let us get the data for the artist
    # then let us extract the setlists into a dataframe for easier filtering/manipulation
    dates = []
    for i in range(len(json_data)):
        date = {}
        date.update({"mbid":json_data[i]["artist"]["mbid"]})
        date.update({"name":json_data[i]["artist"]["name"]})
        date.update({"id":json_data[i]["id"]})
        date.update({"event_date":json_data[i]["eventDate"]})
        date.update({"city":json_data[i]["venue"]["city"]["name"]})
        date.update({"country":json_data[i]["venue"]["city"]["country"]['name']})
        try:
            date.update({"venue":json_data[i]["venue"]["name"]})
        except:
            date.update({"venue":"Unavailable"})
        try:
            if "encore" in json_data[i]["sets"]["set"][-1].keys():
                date.update({"has_encore":True})
            else:
                date.update({"has_encore":False})
        except:
            date.update({"has_encore":False})
        try:
            tot_length = 0
            for x in range(len(json_data[i]["sets"]["set"])):
                tot_length += len(json_data[i]["sets"]["set"][x]["song"])
            date.update({"total_length":tot_length})
        except:
            date.update({"total_length":0})
        dates.append(date)
    dates_df = pd.DataFrame(dates)

    # make the event date into right format
    dates_df["event_date"] = pd.to_datetime(dates_df["event_date"], format="%d-%m-%Y")

    # let us identify the most recent tour where they stopped at the salt shed
    dates_df = dates_df.sort_values("event_date").reset_index(drop=True)
    dates_df["time_diff"] = dates_df["event_date"].diff()
    dates_df.loc[0, "time_diff_group"] = 0
    dates_df.loc[dates_df["time_diff"].dt.days>=days, "time_diff_group"] = dates_df.reset_index()["index"]
    dates_df["time_diff_group"] = dates_df["time_diff_group"].ffill()
    event_date = dates_df[(dates_df["venue"]==venue)]["time_diff_group"].iloc[-1]

    dates_df_filt = dates_df[# just dates in our date group
                             (dates_df["time_diff_group"]==event_date)
                             # exclude saturday night live and some of the festival locales
                             & ~(dates_df["venue"].isin(["Saturday Night Live", "Zilker Park",
                                                                "Grant Park", "Flushing Meadows Corona Park",
                                                                "Empire Polo Club", "Golden Gate Park",
                                                                "The Tonight Show Starring Jimmy Fallon", "BBC Radio 1's Live Lounge",
                                                                "KCRW Studios", "Fort Adams State Park", "Great Stage Park",
                                                                "Harvard Athletic Complex", "Douglass Park", "Plaines d'Abraham",
                                                                "Harriet Island Regional Park", "Napa Valley Expo", "West Hollywood Park",
                                                                "Parc de la Francophonie", "LeBreton Flats Park", "Avenue of the Saints Amphitheater and Event Center",
                                                                ""]))
                             # english speaking North America dates.... trying to limit different tastes as much as possible
                             & (dates_df["country"].isin(["United States", "Canada"]))
                             # exclude those with very few songs... most headlining sets are over 5 songs
                             & (dates_df["total_length"]>5)
                            ].copy()

    # now let us filter setlists with too little information
    if outlier == "std":
        # print(dates_df_filt["total_length"].mean()-2*(dates_df_filt["total_length"].std()))
        dates_df_filt = dates_df_filt[dates_df_filt["total_length"] >= (
            dates_df_filt["total_length"].mean()-2*(dates_df_filt["total_length"].std())
        )]
    elif outlier == "iqr":
        # print(dates_df_filt["total_length"].quantile(0.25)-1.5*(dates_df_filt["total_length"].quantile(0.75)-dates_df_filt["total_length"].quantile(0.25)))
        dates_df_filt = dates_df_filt[dates_df_filt["total_length"] >= (
            dates_df_filt["total_length"].quantile(0.25)-1.5*(dates_df_filt["total_length"].quantile(0.75)-dates_df_filt["total_length"].quantile(0.25))
        )]
    else:
        ValueError("outlier can be one of std or iqr")

    
    artist_name = json_data[0]["artist"]["name"].lower().replace(" ", "_").replace(".", "")
    os.makedirs(folder, exist_ok=True)
    dates_df_filt[['mbid', 'name', 'id', 'event_date', 'city', 'country', 'venue',
                   'has_encore', 'time_diff', 'time_diff_group', 'total_length']].to_csv(f"{folder}{artist_name}_filter_dates.csv", index=False)

    return dates_df_filt

def song_artists(json_data, dates_df_filt, folder="../data/processed/salt_shed_artists/", pct=0.95):
    # now let us make the song data frames
    setlists_df = []
    for set in json_data:
        if set["id"] in dates_df_filt["id"].unique():
            set_data = {}
            set_data.update({"id":set["id"],
                             "city":set["venue"]["city"]["name"]})
            try:
                set_data.update({"venue":set["venue"]["name"]})
            except:
                set_data.update({"venue":"Unavailable"})
            sets = []
            encores = []
            if len(set["sets"]["set"]) > 0:
                for part in set["sets"]["set"]:
                    if "encore" in part.keys():
                        encores = encores + part["song"]
                    else:
                        sets = sets + part["song"]
                set_data.update({"encore_songs":encores})
                set_data.update({"set_songs":sets})
                setlists_df.append(set_data)
    setlists_df = pd.DataFrame(setlists_df)
    setlists_df_set = setlists_df[["id", "venue", "city","set_songs"]].copy().rename(columns={"set_songs":"songs"})
    setlists_df_set["is_encore"] = False
    setlists_df_encore = setlists_df[["id","venue", "city", "encore_songs"]].copy().rename(columns={"encore_songs":"songs"})
    setlists_df_encore["is_encore"] = True
    setlists_df_exp = pd.concat([setlists_df_set, setlists_df_encore]).reset_index(drop=True).sort_values(["id", "is_encore"])
    setlists_df_exp = setlists_df_exp.explode("songs").copy().dropna()
    setlists_df_exp["songs"] = setlists_df_exp["songs"].apply(lambda x: x["name"])

    # song summary
    song_sum = setlists_df_exp[["id","songs"]].drop_duplicates().groupby("songs", as_index=False).size().sort_values("size", ascending = False)

    # our calculations
    ## percent of tour songs that are played at almost every show (n-1 and above)
    try:
        pct_tour_songs = song_sum[song_sum["size"]>=min(setlists_df_exp["id"].nunique()-1, np.floor(setlists_df_exp["id"].nunique()*pct))].shape[0]/song_sum.shape[0]
    except:
        pct_tour_songs = None

    ## average percent of songs played at the show that are part of the base of songs performed at nearly every show ()
    setlists_df_exp["is_base"] = setlists_df_exp["songs"].isin(song_sum.loc[song_sum["size"]>=min(setlists_df_exp["id"].nunique()-1, np.floor(setlists_df_exp["id"].nunique()*pct)), "songs"])
    setlists_df_exp["base_song_pct"] = setlists_df_exp.groupby("id")["is_base"].transform("sum")/ setlists_df_exp.groupby("id").transform("size")
    pct_setlist_songs = setlists_df_exp[["id", "base_song_pct"]].drop_duplicates()["base_song_pct"].mean()

    ## write out our datasets
    artist_name = json_data[0]["artist"]["name"].lower().replace(" ", "_").replace(".", "")

    # our song summary
    song_sum.to_csv(f"{folder}{artist_name}_song_summary.csv", index=False)
    setlists_df_exp.to_csv(f"{folder}{artist_name}_setlists.csv", index=False)

    return {"artist_mbid":json_data[0]["artist"]["mbid"],
            "artist_name":json_data[0]["artist"]["name"],
            "first_tour_date":dates_df_filt["event_date"].min(),
            "pct_tour_songs":pct_tour_songs,
            "pct_setlist_songs":pct_setlist_songs,
            "tour_date_num":dates_df_filt.shape[0],
            "avg_set_length":dates_df_filt["total_length"].mean()}

In [4]:
errors = []
more_data_needed = []
artist_info = []
for file in files:
    try:
        with open(f"{directory}{file}", 'r') as json_file:
            c = json_file.read()
        data = json.loads(c)
    except:
        errors.append(file)
        continue
    try:
        dates_df_filt = clean_dates(data, days=31, outlier="iqr")
        if dates_df_filt.shape[0]>0:
            info = song_artists(data, dates_df_filt, pct=0.9)
            artist_info.append(info)
        else:
            print(f"Not enough data for {file.replace('_raw_data.json', '')}")
    except:
        more_data_needed.append(file)

# add mk.gee
file = "../data/source/mk_gee/mk_gee_raw_data.json"
try:
    with open(file, 'r') as json_file:
        c = json_file.read()
    data = json.loads(c)
except:
    errors.append(file)
try:
    dates_df_filt = clean_dates(data, days=31, outlier="iqr", venue="Vic Theatre")
    if dates_df_filt.shape[0]>0:
        info = song_artists(data, dates_df_filt, pct=0.9)
        artist_info.append(info)
    else:
        print(f"Not enough data for {file.replace('_raw_data.json', '')}")
except:
    more_data_needed.append(file)

Not enough data for lil_dicky


In [5]:
more_data_needed

[]

In [6]:
artist_info_df = pd.DataFrame(artist_info)

In [7]:
artist_info_df_filt = artist_info_df[(artist_info_df["tour_date_num"]>=10)
                                     &(artist_info_df["avg_set_length"]>=10)].sort_values("pct_setlist_songs")
artist_info_df_filt

,artist_mbid,artist_name,first_tour_date,pct_tour_songs,pct_setlist_songs,tour_date_num,avg_set_length
99,84a69823-3d4f-4ede-b43f-17f85513181a,Joe Russo's Almost Dead,2025-08-08,0.000000,0.000000,18,14.333333
6,cff95140-6d57-498a-8834-10eb72865b29,The String Cheese Incident,2025-01-31,0.000000,0.000000,11,17.090909
31,ea5883b7-68ce-48b3-b115-61746ea53b8c,My Morning Jacket,2025-10-08,0.000000,0.000000,16,22.062500
27,822b1e81-d1a0-47fb-b7f9-ef614b913603,Trey Anastasio Band,2024-05-09,0.000000,0.000000,11,18.818182
92,199596a3-a1af-49f8-8795-259eff8461fb,Greensky Bluegrass,2023-06-24,0.000000,0.000000,70,18.000000
...,...,...,...,...,...,...,...
73,a66999a7-ae5c-460e-ba94-1a01143ae847,Snow Patrol,2025-03-21,0.894737,0.988095,14,17.142857
101,d9a9b6aa-c5c5-40dc-95ec-0bff029245c0,Dayseeker,2024-09-27,0.875000,0.995000,25,14.080000
135,a7409219-a681-4072-adb2-5285106ce6f2,The The,2024-10-11,0.931034,0.995536,16,27.125000
35,20883363-1ea4-4d72-ad72-c0e767038f3e,The Mars Volta,2025-10-25,1.000000,1.000000,21,18.000000


In [8]:
artist_info_df_filt.loc[artist_info_df_filt["pct_setlist_songs"]!=0,"pct_setlist_songs"].mean()

np.float64(0.7164233741602374)

In [9]:
artist_info_df_filt["pct_setlist_songs"].median()

np.float64(0.7944950884837365)

In [10]:
# write out artist info
artist_info_df_filt.to_csv("../data/processed/artist_info_filtered.csv", index=False)